# Generation of Prompt Summaries with CarperAI Summarization Model

## Preparations

In [1]:
#!pip install torch torchvision torchaudio pandas

In [130]:
# imports !!! am ende nochmal durchschauen, welche wirklich gebraucht werden !!!

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm
import time

from datasets import load_dataset
import matplotlib.pyplot as plt
#import spacy
#import eng_spacysentiment
import numpy as np
import pandas as pd
from collections import Counter
from ast import literal_eval
from scipy.stats import wilcoxon

## Load Matching and Mismatching Prompts

In [129]:
# load matches and mismatches
match_prompts = pd.read_csv("match_prompts.csv")
mismatch_prompts = pd.read_csv("mismatch_prompts.csv")

In [134]:
# rename columns in the way polina needs them
match_prompts = match_prompts.rename(columns={'Unnamed: 0': 'prompt_id', 'prompt': 'text'})
mismatch_prompts = mismatch_prompts.rename(columns={'Unnamed: 0': 'prompt_id', 'prompt': 'text'})

## Predict summarizations by feeding selected prompts in summarization model

In [135]:
# load tokenizer and summarization model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
#model = AutoModelForCausalLM.from_pretrained("CarperAI/openai_summarize_tldr_ppo")

In [136]:
# function to generate summaries given the dataset with sample prompts

def generate_summaries(prompts_df):
    
    outputs = []

    for text in prompts_df['text']:
        
        # tokenize prompt
        input = tokenizer(text, return_tensors="pt")
        
        # Initialize tqdm manual progress bar
        pbar = tqdm(total=1, desc="Generating summaries")

        # Record the start time
        start_time = time.time()

        # Generate output
        output = 'test summary' # model.generate(inputs['input_ids'], max_new_tokens=100, do_sample=False)
        outputs.append(output)

        # Record the end time
        end_time = time.time()

        # Update tqdm progress bar to complete
        pbar.update(1)

        # Close the tqdm bar
        pbar.close()

        # Calculate and print the time taken
        time_taken = end_time - start_time
        print(f"Time taken to generate summary: {time_taken:.2f} seconds")

    prompts_df['summary'] = outputs
    return prompts_df

In [137]:
# generate summaries for matching and mismatching test prompts
summaries_df_match = generate_summaries(match_prompts)
summaries_df_mismatch = generate_summaries(mismatch_prompts)

Generating summaries:   0%|          | 0/1 [00:00<?, ?it/s]

Time taken to generate summary: 0.00 seconds


Generating summaries:   0%|          | 0/1 [00:00<?, ?it/s]

Time taken to generate summary: 0.00 seconds


Generating summaries:   0%|          | 0/1 [00:00<?, ?it/s]

...

Time taken to generate summary: 0.00 seconds


Generating summaries:   0%|          | 0/1 [00:00<?, ?it/s]

Time taken to generate summary: 0.00 seconds


Generating summaries:   0%|          | 0/1 [00:00<?, ?it/s]

Time taken to generate summary: 0.00 seconds


In [121]:
# concatenate matching and mismatching summary datasets
summaries_df = pd.concat([summaries_df_match, summaries_df_mismatch], ignore_index=True)
summaries_df

,prompt_id,text,summary
0,415,SUBREDDIT: r/relationship_advice\nTITLE: My [M...,test summary
1,440,"SUBREDDIT: r/AskReddit\nTITLE: Reddit, how do ...",test summary
2,466,SUBREDDIT: r/relationships\nTITLE: My siblings...,test summary
3,571,SUBREDDIT: r/dogs\nTITLE: [Help] Older dog has...,test summary
4,627,SUBREDDIT: r/relationship_advice\nTITLE: I [20...,test summary
...,...,...,...
99,6054,SUBREDDIT: r/tifu\nTITLE: TIFU by attempting t...,test summary
100,6310,SUBREDDIT: r/relationships\nTITLE: Me(21f) and...,test summary
101,6367,SUBREDDIT: r/offmychest\nTITLE: someone stole ...,test summary
102,6371,SUBREDDIT: r/self\nTITLE: i don't give a rat's...,test summary


In [138]:
# save df in csv
summaries_df.to_csv("text_summaries.csv")